<a href="https://colab.research.google.com/github/ImPavloh/WhiTTsper-The-Lora/blob/main/WhiTTsper_the_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **WhiTTsper the Lora**

(Whisper + TTs + Alpaca-Lora) - *No soy bueno con los nombres*

**Recuerda cambiar el tipo de entorno de ejeccuión a GPU antes de ejecutar el código.**

**Instalar librerías y dependencias**
 
*~2 minutos de instalación*

In [ ]:
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q datasets loralib sentencepiece
!pip install -q bitsandbytes
!pip install -q langdetect
!pip install -q gradio
!pip install -q torch
!pip install -q gtts
import torch
import whisper
import tempfile
import gradio as gr
from gtts import gTTS
from peft import PeftModel
from langdetect import detect
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

Cargar y configurar el modelo Alpaca-lora (LLaMA) de 7B parámetros.

*~3 minutos de ejecución*

In [ ]:
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
modelo = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
modelo = PeftModel.from_pretrained(modelo, "tloen/alpaca-lora-7b")

Definir *funciones* que permiten construir un **sistema de preguntas y respuestas basado en tareas**.

In [ ]:
def gen_instruccion(instruccion, input=None):
    input_section = f"### Entrada:\n{input}\n" if input else ""
    return f"""A continuación se muestra una instrucción que describe una tarea. Escribe una respuesta que complete adecuadamente la petición.

### Instrucción:
{instruccion}

{input_section}### Respuesta:"""

generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
    num_return_sequences=1,
    max_length=512,
)

def evaluate(instruction, input=None):
    prompt = gen_instruccion(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(modelo.device)
    generation_output = modelo.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    sequence = generation_output.sequences[0]
    output = tokenizer.decode(sequence)
    response_text = output.split("### Respuesta:")[-1].strip()
    return response_text

# **Interfaz Gradio**

Al hacer clic en el botón de grabación, el micrófono comenzará a grabar. El usuario tiene también la opción de seleccionar el tamaño del modelo de Whisper. 

Cuanto más grande sea el modelo de Whisper mejor será pero tardará más.

1.   **Tiny** 39 M parámetros
2.   **Base** 74 M parámetros **[recomendado]**
3.   **Small** 244 M parámetros
4.   **Medium** 269 M parámetros
5.   **Large** 1550 M parámetros



Una vez que se envía la grabación, se transcribirá automáticamente el audio con Whisper y LLaMa responderá con un mensaje. El mensaje estará en formato de texto y audio, junto con la transcripción del audio original.

In [ ]:
def stt(tmp_filename, model_size):
    try:
    # Carga el modelo de Whisper
        modelo = whisper.load_model(model_size)

    # Transcribe el audio a texto y después genera una respuesta
        result = modelo.transcribe(tmp_filename)

        text_input = result['text']
        response_text = evaluate(text_input)

    # Genera respuesta de voz usando Google TTS y la guarda
        idioma = detect(response_text)
        tts = gTTs(response_text, lang=idioma)
        _, response_audio_path = tempfile.mkstemp(suffix='.mp3')
        tts.save(response_audio_path)

        return text_input, response_audio_path, response_text

    except (ValueError, whisper.exceptions.AudioProcessingError, Exception) as e:
        error_message = str(e)

        return f"Error: {error_message}", None, None

# Interfaz de Gradio
inputs = [
    gr.Audio(source='microphone', type='filepath', label="Habla aquí 🗣️"),
    gr.Dropdown(choices=['tiny', 'base', 'small', 'medium', 'large'], value='base', label="Tamaño del modelo 📦"),
]

outputs = [
    gr.Textbox(label="Transcripción de tu input 📝"),
    gr.Audio(type='filepath', label="Respuesta 🔊"),
    gr.Textbox(label="Respuesta 📄"),
]

pavdemo = gr.Interface(
    fn=stt,
    inputs=inputs,
    outputs=outputs,
    title="WhiTTsper the Lora",
    description="Esta demo utiliza tecnologías de reconocimiento de voz y síntesis de voz, Whisper de OpenAI y Google Text-to-Speech respectivamente, para interactuar con la IA Alpaca-LoRA ~ modelo LLaMa 7B ).",
).launch()